In [41]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import csv

from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import asyncio
import time
import aiohttp

INITIATE PROXY POOL

CODE TEST PROXY

In [43]:
# # define custom options for the Selenium driver
# options = Options()

# options.add_argument(f'--proxy-server={proxy_pool[7]}')

# # create the ChromeDriver instance with custom options
# driver = webdriver.Chrome(options=options)

# driver.get('http://httpbin.org/ip')
# # driver.get('https://www.malt.fr/profile/luccharlopeau')

BACKUP CSV CODE 

In [44]:
async def save_dataframe_periodically(df_scraped):
    while True:
        await asyncio.sleep(3600)  # Sleep for one hour (3600 seconds)
        df_scraped.to_csv('scraped_data.csv', index=False)  # Save the DataFrame to a CSV file

In [45]:
# function that gets a df and gives the next profile to scrap, when its scrapped it updates the df
def get_next_profile(df):
    # get the first profile that is not scrapped
    next_profile = df[df['scraped']==False].iloc[0]
    # update the df
    df.loc[df['profil']==next_profile['profil'], 'scraped'] = True
    return df, next_profile

SCRAP USER FUNCTION

In [46]:
async def scrap_user(row, driver):
    wait = WebDriverWait(driver, 5)
    driver.get(row['link'])
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    data = {}

    price_element = soup.find('div', {'data-testid': 'profile-price'})
    if price_element:
        price = price_element.find('span', class_='block-list__price').text
        data['price'] = price.strip()
    
    # Récupérer l'expérience
    experience_element = soup.find('span', string='Expérience')
    if experience_element:
        experience = experience_element.find_next('span', class_='profile-indicators-content').text
        data['experience'] = experience.strip()
    
    # Récupérer le taux de réponse
    response_rate_element = soup.find('span', string='Taux de réponse')
    if response_rate_element:
        response_rate = response_rate_element.find_next('span', class_='profile-indicators-content').text
        data['response_rate'] = response_rate.strip()
    
    # Récupérer le temps de réponse
    response_time_element = soup.find('span', string='Temps de réponse')
    if response_time_element:
        response_time = response_time_element.find_next('span', class_='profile-indicators-content').text
        data['response_time'] = response_time.strip()
        
    # Récupérer le nom 
    name_element = soup.find('div', {'data-testid': 'profile-fullname'})
    if name_element:
        name = name_element.text
        data['name'] = name.strip()
        
    # Récupérer le métier
    headline_element = soup.find('div', {'data-testid': 'profile-headline'})
    if headline_element:
        headline = headline_element.text
        data['headline'] = headline.strip()
        
    # Récupérer le nombre de missions
    missions_element = soup.find('div', {'data-testid': 'profile-counter-missions'})
    if missions_element:
        missions = missions_element.find('strong').text
        data['missions'] = missions.strip()
        
    # Récupérer toutes les catégories
    categories_elements = soup.find_all('li', {'class': 'categories__list-item'})
    categories = [category.find('a').text for category in categories_elements]
    data['categories'] = categories
    
    
    # Récupérer les compétences
    competences_element = soup.find_all('div', {'class': 'profile-expertises__content-list-item__label'})
    competences = [competence.find('a', class_='joy-link joy-link_teal').text.strip() for competence in competences_element]

    data['competences'] = competences
    
    # Récupérer le statut "Supermalter"
    supermalter_element = soup.find('span', class_='joy-badge-level__tag blue')
    if supermalter_element:
        supermalter = supermalter_element.get_text(strip=True)
        data['supermalter'] = supermalter
        
    # Récupérer la localisation
    location_element = soup.find('dl', {'class': 'profile__location-and-workplace-preferences__item'})
    if location_element:
        location_label = location_element.find('dt', {'data-testid': 'profile-location-address-label'})
        location_value = location_element.find('dd', {'data-testid': 'profile-location-preference-address'})

        if location_label and location_value:
            location = {location_label.text: location_value.text}
            data['location'] = location
            
    # Récupérer la préférence de télétravail
    teletravail_element = soup.find('dl', {'class': 'profile-page-mission-preferences__item'})
    if teletravail_element:
        teletravail_label = teletravail_element.find('dt')
        teletravail_value = teletravail_element.find('dd')

        if teletravail_label and teletravail_value:
            teletravail_preference = {teletravail_label.text: teletravail_value.text}
            data['teletravail_preference'] = teletravail_preference
            
    # Récupérer le nombre de recommandations
    recommendations_element = soup.find('span', {'data-testid': 'profile-counter-recommendations'})
    if recommendations_element:
        recommendations_count = int(recommendations_element.text.split()[0])
        data['recommendations'] = recommendations_count   
        

    # Récupérer le message de présentation
    presentation_element = soup.find('div', {'class': 'profile-description__content'})
    if presentation_element:
        presentation_message = presentation_element.get_text(strip=True)
        data['presentation'] = presentation_message
        
    # add link of the profile
    data['link'] = row['link']
    
    # add created date
    data['creation_date'] = row['creation_date']
    
    # add name to the data
    data['profil'] = row['profil']
        
    driver.quit() # close the browser
    # time.sleep(5) # wait for 5 seconds to avoid getting banned
    
    print(data)
    
    return data # return the data scrapped

In [47]:
import asyncio
from selenium import webdriver
import threading

async def configure_webdriver(proxy_address):
    # define custom options for the Selenium driver
    options = Options()

    options.add_argument(f'--proxy-server={proxy_address}')
    # options.add_argument("--headless") # breaks the code
    options.add_argument("window-size=400,200")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("start-maximized")
    options.add_argument("enable-automation")
    options.add_argument("--disable-infobars")
    options.add_argument("--disable-dev-shm-usage")

    # create the ChromeDriver instance with custom options
    driver = webdriver.Chrome(options=options)
    
    return driver

async def scrap_all_users_proxy(proxy_addresses, df):
    tasks = []
    index = 1
    df_scraped = pd.DataFrame()
    
    # # save the df every hour
    # loop = asyncio.get_event_loop()
    # loop.create_task(save_dataframe_periodically(df_scraped))
    
    while df[df['scraped']==False].shape[0] > 0: # while there are profiles to scrap
        for proxy_address in proxy_addresses: # for each proxy address
            driver = await configure_webdriver(proxy_address) # configure the driver with the proxy address
            df, row = get_next_profile(df) # get the next profile to scrap
            task = asyncio.create_task(scrap_user(row, driver)) # scrap the profile in a task (async) and save the result in a variable 
            tasks.append(task)
            
        results = await asyncio.gather(*tasks) # wait for all the tasks to finish
        
        

        print(results)
        print(f'Fetched ', index, 'to', index+10, ' profiles')
        index += 10
        
        # update the df
        
        #save every hour
    

RUN THE SCRIPT WITH THE LINKS

In [48]:
profile_links = pd.read_csv('../data/links.csv')
profile_links['profil'] = profile_links['profil'].apply(lambda x: x.replace('https://www.malt.fr/profile/', ''))

# add column link to the DataFrame
profile_links['link'] = profile_links['profil'].apply(lambda x: f'https://www.malt.fr/profile/{x}')

profile_links['scraped'] = False # add column scraped to the DataFrame

# get first 10 rows of the DataFrame
df = profile_links.iloc[:30]

async def main():
    return await scrap_all_users_proxy(proxy_pool, df)

data = await main()
data

{'categories': [], 'competences': [], 'link': 'https://www.malt.fr/profile/bricetillet', 'creation_date': '2014-01-08', 'profil': 'bricetillet'}
{'categories': [], 'competences': [], 'link': 'https://www.malt.fr/profile/mickaelmolina', 'creation_date': '2013-08-02', 'profil': 'mickaelmolina'}
{'price': '700\xa0€', 'experience': '-', 'response_rate': '100%', 'response_time': '1h', 'name': 'Alexandre Labayle', 'headline': 'Consultant décisionnel Senior', 'categories': [], 'competences': [], 'location': {'Localisation': ''}, 'teletravail_preference': {'Peut travailler dans vos locaux à': ' et 50km autour'}, 'presentation': "Consultant Senior en Business Intelligence , 14 ans d'expérience.Certifié SAP Business Objects et Microsoft BI. Missions d'architecture technique (cluster SAP BI ,implémentation de sécurité). Modélisation et alimentation de DatawareHouse (Dataservices / SSIS/SSAS) . Création de rapports Excel PowerPivot/Sap Business Objects .Création d'univers ou de cubes SSAS.", 'link

WebDriverException: Message: unknown error: net::ERR_TUNNEL_CONNECTION_FAILED
  (Session info: chrome=119.0.6045.159)
Stacktrace:
#0 0x5572a51e76d4 <unknown>
#1 0x5572a4eec48e <unknown>
#2 0x5572a4ee59e9 <unknown>
#3 0x5572a4ed8602 <unknown>
#4 0x5572a4ed9d05 <unknown>
#5 0x5572a4ed8b88 <unknown>
#6 0x5572a4ed7999 <unknown>
#7 0x5572a4ed77ff <unknown>
#8 0x5572a4ed65ba <unknown>
#9 0x5572a4ed6a15 <unknown>
#10 0x5572a4eef265 <unknown>
#11 0x5572a4f6b441 <unknown>
#12 0x5572a4f54132 <unknown>
#13 0x5572a4f6af65 <unknown>
#14 0x5572a4f53ed3 <unknown>
#15 0x5572a4f26420 <unknown>
#16 0x5572a4f27a93 <unknown>
#17 0x5572a51ba4c0 <unknown>
#18 0x5572a51bd780 <unknown>
#19 0x5572a51bd1fa <unknown>
#20 0x5572a51bdc95 <unknown>
#21 0x5572a51ac65b <unknown>
#22 0x5572a51be080 <unknown>
#23 0x5572a5197830 <unknown>
#24 0x5572a51d7ee7 <unknown>
#25 0x5572a51d80f5 <unknown>
#26 0x5572a51e6cce <unknown>
#27 0x7f3d586aa9eb <unknown>


In [ ]:
data

SAVE THE DATA TO CSV

In [ ]:
def save_to_csv(data, csv_filename):
    # Assurez-vous que la liste de données n'est pas vide
    if not data:
        print("Aucune donnée à enregistrer.")
        return

    # Créez une liste de noms de colonnes pour le CSV en incluant les nouveaux éléments
    fieldnames = ['name', 'headline', 'price', 'experience', 'response_rate', 'response_time', 'missions', 'categories', 'competences', 'supermalter', 'location','presentation', 'recommendations', 'teletravail_preference']

    # Ouvrir le fichier CSV en mode écriture
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

        # Écrire les en-têtes
        writer.writeheader()

        # Écrire les données
        for entry in data:
            writer.writerow(entry)

    print(f"Données enregistrées dans {csv_filename}")

In [ ]:
csv_filename = 'malt_data.csv'
save_to_csv(data, csv_filename)

Aucune donnée à enregistrer.
